In [1]:
import numpy as np
import numba
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()


Loading BokehJS ...

In [2]:
import electric

In [37]:


@numba.jit(nopython=True)
def run_eval(params,params_ctrl):
    N_steps = int(50000)
    N_states = 4
    x = np.zeros((N_states,1))
    f = np.zeros((N_states,1))
    X = np.zeros((N_states,N_steps))
    V = np.zeros((3,1),dtype=np.complex128)
    out = np.zeros((16,N_steps))
    #self.out_cplx = np.zeros((72,N_steps),np.complex128)
    


    T = np.zeros((N_steps,1))
       
    V = np.zeros((4,1),dtype=np.complex128)
    V[0] = 325*np.exp(1j*np.deg2rad(0.0))   
    V[1] = 325*np.exp(1j*np.deg2rad(-120.0))   
    V[2] = 300*np.exp(1j*np.deg2rad(-240.0)) 
    V[3] = 0.0*np.exp(1j*np.deg2rad(-240.0)) 
    
    t = 0.0

    params[0].v_abcn[:]  = electric.ph2inst(np.array([t]),V) 
    electric.vsc_emag(0.0,0,params,params_ctrl)
    electric.vsc_emag(0.0,1,params,params_ctrl)    
    
    dt = params[0].Dt_ps
    
    x[:] = params[0].x
    X[:,0] = x[:,0]
    T[0,0] = t
    for it_step in range(1,N_steps):  
        t += dt
        
        #if t>0.2: params_ctrl[0].i_dq0_ref[1] =  200.0
        if t>0.1: params_ctrl[0].S_ref[:] =  -np.ones((3,1)) * 1j*1.0e3/3
        #if t>0.2: params_ctrl[0].v_dc_ref = 1000.0
        #if t>0.3: params_ctrl[0].i_dq0_ref[1] = -100.0
        #if t>0.2: params[0].i_dc =  0.1
        #if t>2.0: params[0].i_dc =  0.2
            
        params[0].v_abcn[:] = electric.ph2inst(np.array([t]),V)
         
        params[0].x[:] = x
        electric.vsc_emag(t,1,params,params_ctrl)   
        
        f[:] = params[0].f 

        x = x + dt*f
        
        electric.vsc_emag(t,2,params,params_ctrl)   
        
        X[:,it_step] = x[:,0]
        out[0:4,it_step] = params[0].v_abcn[:,0]
        out[4:8,it_step] = params[0].i_abcn[:,0]
        out[8:12,it_step] =params[0].eta_abcn[:,0]
        out[12:15,it_step] =params_ctrl[0].i_dq0[:,0]
        T[it_step,0] = t
            
    return T,X,out
#%timeit run_eval(params,params_ctrl)

In [40]:
vsc_objs = electric.VSC_emag('vsc_feed_1bus.json')
t=np.array([0.0])
mode = 1
params = vsc_objs.params_vsc
params_ctrl = vsc_objs.params_ctrl_vsc
vsc_objs.ctrl_design()
print(params_ctrl[0].K_p_vdc)
params_ctrl[0].T_pi_vdc = 10.0
T,X,out = run_eval(params,params_ctrl)
decim = 10
p = np.sum(out[0:4,0:-1:decim]*out[4:8,0:-1:decim],axis=0)

0.44


In [41]:
fig = figure(width=600, height=300, title='v_abcn')
decim = 10
fig.line(T[0:-1:decim,0],out[0,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],out[1,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],out[2,0:-1:decim], line_width=1, color='blue')  
show(fig)

fig = figure(width=600, height=300, title='i_abcn')
decim = 10
fig.line(T[0:-1:decim,0],out[4,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],out[5,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],out[6,0:-1:decim], line_width=1, color='blue')  
show(fig)

fig = figure(width=600, height=300, title='eta_abcn')
decim = 10
fig.line(T[0:-1:decim,0],out[8,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],out[9,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],out[10,0:-1:decim], line_width=1, color='blue')  
show(fig) 

fig = figure(width=600, height=300, title='p')
decim = 10
fig.line(T[0:-1:decim,0],p, line_width=1, color='red')
show(fig)

fig = figure(width=600, height=300, title='i_dq0')
decim = 10
fig.line(T[0:-1:decim,0],out[12,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],out[13,0:-1:decim], line_width=1, color='green')
show(fig)

fig = figure(width=600, height=300,  title='v_dc')
decim = 10
fig.line(T[0:-1:decim,0],X[2,0:-1:decim], line_width=1, color='blue')
show(fig)

In [33]:
import ctrl
decim = 1
v_abc = out[0:3,0:-1:decim]
i_abc = out[4:7,0:-1:decim]

In [7]:
v_dq_pos = ctrl.park2(v_abc,T[:,0], 2*np.pi*50)
v_dq_neg = ctrl.park2(v_abc,T[:,0],-2*np.pi*50)

i_dq_pos = ctrl.park2(i_abc,T[:,0], 2*np.pi*50)
i_dq_neg = ctrl.park2(i_abc,T[:,0],-2*np.pi*50)

In [8]:
fig = figure(width=600, height=300, title='eta_abcn')
decim = 10
fig.line(T[0:-1:decim,0],dq[0,:].real, line_width=1, color='red')
fig.line(T[0:-1:decim,0],dq[1,:].real, line_width=1, color='green')
#fig.line(T[0:-1:decim,0],out[9,0:-1:decim], line_width=1, color='green')    
#fig.line(T[0:-1:decim,0],out[10,0:-1:decim], line_width=1, color='blue')  
show(fig) 

NameError: name 'dq' is not defined

In [ ]:
N = fft.shape[0]
v_fft_pos = np.fft.fft(v_dq_pos[0,:]+1j*v_dq_pos[1,:])
v_fft_neg = np.fft.fft(v_dq_neg[0,:]+1j*v_dq_neg[1,:])
i_fft_pos = np.fft.fft(i_dq_pos[0,:]+1j*i_dq_pos[1,:])
i_fft_neg = np.fft.fft(i_dq_neg[0,:]+1j*i_dq_neg[1,:])
freqs = np.fft.fftfreq(N, (T[1]-T[0])*decim)

In [ ]:
fig = figure(width=600, height=300, title='eta_abcn')
decim = 10
fig.line(freqs,np.abs(v_fft_pos)/N, line_width=1, color='red')
fig.line(freqs,np.abs(v_fft_neg)/N, line_width=1, color='green')
#fig.line(freqs,fft.imag, line_width=1, color='green')
#fig.line(T[0:-1:decim,0],dq[1,:].real, line_width=1, color='green')
#fig.line(T[0:-1:decim,0],out[9,0:-1:decim], line_width=1, color='green')    
#fig.line(T[0:-1:decim,0],out[10,0:-1:decim], line_width=1, color='blue')  
show(fig) 

In [ ]:
V_pos = v_fft_pos[freqs==0]/N
V_neg = v_fft_neg[freqs==0]/N
I_pos = i_fft_pos[freqs==0]/N
I_neg = i_fft_neg[freqs==0]/N

In [ ]:
I_pos